#Mangrove Loss Mapping in Asia

This code is for create a map of loss in mangrove forest in Asia using hexagons

In [ ]:
from pyproj import CRS
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
hexagons = gpd.read_file('/content/drive/MyDrive/colab_data/Mangrove/GMW_v3/hexagon_grid_asia/hexgrid_Asia_10km_join.shp')


In [ ]:
hexagons['cumulative_loss'] = 0  # Initialize a new column for cumulative loss
hexagons.reset_index(inplace=True, drop=False) # Reset index and keep it as a column

folder_path = '/content/drive/MyDrive/colab_data/Mangrove/GMW_v3/Changes'
year_pairs = [("1996", "2007"), ("2007", "2008"), ("2008", "2009"), ("2009", "2010"), ("2010", "2015"), ("2015", "2016"), ("2016", "2017"), ("2017", "2018"), ("2018", "2019"), ("2019", "2020")]

for year_pair in year_pairs:
    file_path = f"{folder_path}/gmw_asia_{year_pair[0]}_{year_pair[1]}_Union.shp"
    mangrove_data = gpd.read_file(file_path)
    loss_data = mangrove_data[mangrove_data['Change'] == 'Loss']

    # Spatial join with the hexagon grid using 'predicate' instead of 'op'
    joined_data = gpd.sjoin(hexagons, loss_data, how="inner", predicate='intersects')

    # Calculate the area of loss polygons
    joined_data['loss_area'] = joined_data.geometry.area/1000000

    # Sum the loss area for each hexagon using the 'index' column from hexagons
    loss_by_hexagon = joined_data.groupby('index')['loss_area'].sum().reset_index()

    # Update the cumulative loss column in the hexagons DataFrame
    hexagons['cumulative_loss'] += hexagons['index'].map(loss_by_hexagon.set_index('index')['loss_area']).fillna(0)

In [ ]:



output_path = "/content/drive/MyDrive/colab_data/Mangrove/GMW_v3/hexagons_output.gpkg"

hexagons.to_file(output_path, driver="GPKG")

